# Transformamos datos

In [2]:
import pandas as pd
from pathlib import Path

file_path = '../data/interim/df_fleca'
df_fleca = pd.read_parquet(file_path)
df_fleca.head()


,fecha,familia,unidades_vendidas,base_imponible,total
0,2023-11-09,PAN,1.0,NaN,NaN
1,2023-11-07,PAN,1.0,NaN,NaN
2,2023-11-08,PAN,1.0,NaN,NaN
3,2023-07-24,PAN,0.8,0.06,0.06
4,2023-08-13,PAN,1.0,0.08,0.08


In [3]:
# Vemos las estadísticas descriptivas de las columnas numéricas
df_fleca.describe(include= 'all')

,fecha,familia,unidades_vendidas,base_imponible,total
count,295979,295978,295979.000000,293732.000000,293732.000000
unique,NaN,16,NaN,NaN,NaN
top,NaN,CAFES,NaN,NaN,NaN
freq,NaN,114569,NaN,NaN,NaN
mean,2024-01-12 01:52:50.037063168,NaN,1.205626,2.002429,2.188847
min,2023-01-02 00:00:00,NaN,0.000000,0.000000,0.000000
25%,2023-07-10 00:00:00,NaN,1.000000,1.320000,1.400000
50%,2023-12-27 00:00:00,NaN,1.000000,1.550000,1.700000
75%,2024-07-25 00:00:00,NaN,1.000000,2.270000,2.500000
max,2025-02-28 00:00:00,NaN,93.000000,287.270000,316.000000


In [4]:
# Revisamos la estructura y tipos de datos
df_fleca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295979 entries, 0 to 295978
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   fecha              295979 non-null  datetime64[ns]
 1   familia            295978 non-null  object        
 2   unidades_vendidas  295979 non-null  float64       
 3   base_imponible     293732 non-null  float64       
 4   total              293732 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 11.3+ MB


In [5]:
# Imputar valores nulos en las columnas 'total' y 'base_imponible'

def imputar_valores_nulos(df: pd.DataFrame, meses_cercanos: list[str], mes_objetivo: str) -> pd.DataFrame:
    """
    Imputa valores nulos en las columnas 'total' y 'base_imponible' utilizando la media
    de los meses más cercanos y, si es necesario, la media anual por familia.

    Args:
        df (pd.DataFrame): DataFrame con las columnas 'fecha', 'familia', 'total' y 'base_imponible'.
        meses_cercanos (list[str]): Lista de meses cercanos (en formato 'YYYY-MM') para calcular la media.
        mes_objetivo (str): Mes objetivo (en formato 'YYYY-MM') donde se imputarán los valores nulos.

    Returns:
        pd.DataFrame: DataFrame con los valores nulos imputados.
    """
    # Validación de columnas requeridas
    required_columns = {'fecha', 'familia', 'total', 'base_imponible'}
    if not required_columns.issubset(df.columns):
        raise ValueError(f"El DataFrame debe contener las columnas: {required_columns}")
    
    # Asegurar que 'fecha' es de tipo datetime
    if not pd.api.types.is_datetime64_any_dtype(df['fecha']):
        df['fecha'] = pd.to_datetime(df['fecha'])
    
    # Copiar el DataFrame para evitar modificar el original
    df = df.copy()
    
    # Convertir 'fecha' a Periodo (mes-año) para facilitar la agrupación
    df['mes'] = df['fecha'].dt.to_period('M')
    
    # Lista de columnas a imputar
    columnas = ['total', 'base_imponible']
    
    # Imputar valores nulos utilizando la media de meses cercanos
    for col in columnas:
        for categoria in df['familia'].unique():
            for mes in meses_cercanos:
                # Calcular la media para la categoría y mes actual
                media = df.loc[
                    (df['familia'] == categoria) & (df['mes'] == mes), col
                ].mean()
                # Imputar valores nulos en el mes objetivo para la columna actual
                df.loc[
                    (df['familia'] == categoria) &
                    (df['mes'] == mes_objetivo) &
                    (df[col].isnull()),
                    col
                ] = media
        
        # Para los valores nulos restantes, imputar con la media anual por familia
        mask_obj = df['mes'] == mes_objetivo
        df.loc[mask_obj, col] = df.groupby('familia')[col].transform(
            lambda x: x.fillna(x.mean())
        )
    
    # Mostrar resumen de valores nulos (opcional)
    print("Valores nulos después de la imputación:")
    print(df.isnull().sum())
    
    return df

# Definir los meses cercanos y el mes objetivo
meses_cercanos = ["2023-10", "2023-12"]
mes_objetivo = "2023-11"

# Llamar a la función para imputar valores nulos
df_fleca = imputar_valores_nulos(df_fleca, meses_cercanos, mes_objetivo)


Valores nulos después de la imputación:
fecha                0
familia              1
unidades_vendidas    0
base_imponible       0
total                0
mes                  0
dtype: int64


In [6]:
def eliminar_filas_nulas_familia(df_fleca: pd.DataFrame) -> pd.DataFrame:
    """
    Elimina las filas del DataFrame que contienen valores nulos en la columna 'familia'
    y muestra la información del DataFrame resultante.

    Args:
        df_fleca (pd.DataFrame): DataFrame que contiene la columna 'familia'.

    Returns:
        pd.DataFrame: DataFrame sin filas con valores nulos en la columna 'familia'.
    """
    df_fleca = df_fleca.dropna(subset=['familia'], how='any')
    print("Valores nulos después de eliminar filas:")
    print(df_fleca.isnull().sum())
        
    return df_fleca

# Para ver los resultados:
df_fleca = eliminar_filas_nulas_familia(df_fleca)

Valores nulos después de eliminar filas:
fecha                0
familia              0
unidades_vendidas    0
base_imponible       0
total                0
mes                  0
dtype: int64


In [7]:
#
def procesar_festivos_semana_santa(df: pd.DataFrame) -> pd.DataFrame:
    """
    Procesa el DataFrame agregando indicadores de festivos y Semana Santa, 
    y filtra los outliers en la columna 'total' (excluyendo transacciones de Semana Santa).

    El procesamiento incluye:
      - Definir fechas festivas.
      - Definir rangos para Semana Santa (2023, 2024 y 2025) y combinarlos.
      - Agregar columnas indicando si la fecha (en el índice) es festivo o corresponde a Semana Santa.
      - Marcar las transacciones que ocurren en Semana Santa según la columna 'fecha'.
      - Filtrar los registros con 'total' > 50 que no correspondan a Semana Santa.
      - Mostrar la cantidad de registros eliminados.
      - Limpiar el DataFrame eliminando columnas auxiliares.

    Args:
        df (pd.DataFrame): DataFrame que debe contener al menos la columna 'fecha' (tipo datetime) y 'total'.
                           Se asume que el índice es de tipo DatetimeIndex para la imputación de festivos y Semana Santa.

    Returns:
        pd.DataFrame: DataFrame procesado.
    """
    # Validar que el índice sea de tipo DatetimeIndex, si no, intentar convertir la columna 'fecha' y establecerla como índice
    if not isinstance(df.index, pd.DatetimeIndex):
        if 'fecha' in df.columns:
            df = df.copy()
            df['fecha'] = pd.to_datetime(df['fecha'])
            df.set_index('fecha', inplace=True)
        else:
            raise ValueError("El DataFrame debe tener un índice de tipo DatetimeIndex o una columna 'fecha'.")
    
    # Definir los festivos
    festivos = ['2023-01-01', '2023-12-25', '2024-01-01', '2024-12-25', '2025-01-01']
    festivos = pd.to_datetime(festivos)
    
    # Definir rango de Semana Santa (asumiendo una duración típica)
    semana_santa_2023 = pd.date_range('2023-04-02', '2023-04-09')
    semana_santa_2024 = pd.date_range('2024-03-24', '2024-03-31')
    semana_santa_2025 = pd.date_range('2025-04-13', '2025-04-20')
    semana_santa = semana_santa_2023.union(semana_santa_2024).union(semana_santa_2025)
    
    # Agregar columnas indicando si el índice (fecha) corresponde a Semana Santa y festivos
    df = df.copy()  # Evitar modificar el original
    df['semana_santa'] = df.index.to_series().apply(lambda x: x in semana_santa)
    df['festivo'] = df.index.to_series().apply(lambda x: x in festivos)
    
    # Verificar datos (opcional)
    print("Vista previa del DataFrame con indicadores de festivos y Semana Santa:")
    print(df.head())
    
    # Marcar si la transacción pertenece a Semana Santa según la columna 'fecha'
    # (Se asume que existe una columna 'fecha'; si no, se puede utilizar el índice)
    if 'fecha' in df.columns:
        df['es_semana_santa'] = df['fecha'].isin(semana_santa)
    else:
        # En caso de no tener la columna 'fecha', se utiliza el índice
        df['es_semana_santa'] = df.index.to_series().isin(semana_santa)
    
    # Eliminar valores extremos en 'total' solo si NO son de Semana Santa
    df_filtrado = df[~((df['total'] > 50) & (df['es_semana_santa'] == False))]
    
    # Mostrar la cantidad de registros eliminados
    registros_eliminados = df.shape[0] - df_filtrado.shape[0]
    print(f"Registros eliminados por ser outliers fuera de Semana Santa: {registros_eliminados}")
    
    # Limpiar el DataFrame eliminando la columna auxiliar
    df_filtrado = df_filtrado.drop(columns=['es_semana_santa'])
    
    return df_filtrado

df_fleca = procesar_festivos_semana_santa(df_fleca)

Vista previa del DataFrame con indicadores de festivos y Semana Santa:
           familia  unidades_vendidas  base_imponible     total      mes  \
fecha                                                                      
2023-11-09     PAN                1.0        1.356664  1.362142  2023-11   
2023-11-07     PAN                1.0        1.356664  1.362142  2023-11   
2023-11-08     PAN                1.0        1.356664  1.362142  2023-11   
2023-07-24     PAN                0.8        0.060000  0.060000  2023-07   
2023-08-13     PAN                1.0        0.080000  0.080000  2023-08   

            semana_santa  festivo  
fecha                              
2023-11-09         False    False  
2023-11-07         False    False  
2023-11-08         False    False  
2023-07-24         False    False  
2023-08-13         False    False  
Registros eliminados por ser outliers fuera de Semana Santa: 6


In [ ]:
# Agrupara por semana y sumar las columnas 'total' y 'base_imponible' y 'unidades_vendidas'

def agrupar_por_semana(df: pd.DataFrame) -> pd.DataFrame:
    """
    Agrupa el DataFrame por semana y suma las columnas 'total', 'base_imponible' y 'unidades_vendidas'.

    Args:
        df (pd.DataFrame): DataFrame con las columnas 'fecha', 'total', 'base_imponible' y 'unidades_vendidas'.

    Returns:
        pd.DataFrame: DataFrame agrupado por semana con las sumas de las columnas especificadas.
    """
    # Validar que el índice sea de tipo DatetimeIndex, si no, intentar convertir la columna 'fecha' y establecerla como índice
    if not isinstance(df.index, pd.DatetimeIndex):
        if 'fecha' in df.columns:
            df = df.copy()
            df['fecha'] = pd.to_datetime(df['fecha'])
            df.set_index('fecha', inplace=True)
        else:
            raise ValueError("El DataFrame debe tener un índice de tipo DatetimeIndex o una columna 'fecha'.")
    
    # Agrupar por semana y sumar las columnas especificadas
    df_semana = df.resample('W').sum(numeric_only=True)
    
    return df_semana

df_fleca_semana = agrupar_por_semana(df_fleca)
print("DataFrame agrupado por semana:")
print(df_fleca_semana.head())

DataFrame agrupado por semana:
            unidades_vendidas  base_imponible    total  semana_santa  festivo
fecha                                                                        
2023-01-08            3075.54         5145.61  5627.68             0        0
2023-01-15            2569.91         4020.09  4392.67             0        0
2023-01-22            2771.42         4406.36  4803.98             0        0
2023-01-29            2528.31         4022.09  4397.29             0        0
2023-02-05            2520.70         3952.34  4320.69             0        0


In [18]:
# Almacenar el DataFrame agrupado en un archivo parquet en la carpeta interim (datos intermedios) por semanas

carpeta = Path("C:/Workspace/mlops_fleca_project/data")
(carpeta / "interim").mkdir(parents=True, exist_ok=True)
df_fleca_semana.to_parquet(carpeta / "interim" / "df_fleca_semana.parquet")
print(f"DataFrame agrupado por semana guardado en: {carpeta / 'df_fleca_semana.parquet'}")

DataFrame agrupado por semana guardado en: C:\Workspace\mlops_fleca_project\data\df_fleca_semana.parquet


In [24]:
# Agrupamos por semana y sumamos las columnas 'total', 'base_imponible' y 'unidades_vendidas'
# y reiniciamos el índice
# y convertimos la columna 'fecha' a tipo datetime

def agrupar_por_semana_y_sumar(df: pd.DataFrame) -> pd.DataFrame:
    """
    Agrupa el DataFrame por semana y suma las columnas 'total', 'base_imponible' y 'unidades_vendidas'.
    Reinicia el índice y convierte la columna de semana a tipo datetime.

    Args:
        df (pd.DataFrame): DataFrame con las columnas 'fecha', 'total', 'base_imponible' y 'unidades_vendidas'.

    Returns:
        pd.DataFrame: DataFrame agrupado por semana con las sumas de las columnas especificadas.
    """
    # Agrupar por semana usando el índice DatetimeIndex
    df_grouped = df.resample('W')[['total', 'base_imponible', 'unidades_vendidas']].sum().reset_index()
    df_grouped.rename(columns={'fecha': 'semana'}, inplace=True)
    return df_grouped

df_fleca_semana = agrupar_por_semana_y_sumar(df_fleca)
print("DataFrame agrupado por semana y con columna 'semana' convertida a datetime:")
print(df_fleca_semana.head())

DataFrame agrupado por semana y con columna 'semana' convertida a datetime:
      semana    total  base_imponible  unidades_vendidas
0 2023-01-08  5627.68         5145.61            3075.54
1 2023-01-15  4392.67         4020.09            2569.91
2 2023-01-22  4803.98         4406.36            2771.42
3 2023-01-29  4397.29         4022.09            2528.31
4 2023-02-05  4320.69         3952.34            2520.70


In [33]:
# Almacenar el DataFrame agrupado en un archivo parquet en la carpeta interim (datos intermedios)
output_path = Path('../data/interim/df_fleca_semana')
df_fleca_semana.to_parquet(output_path, index=False)   

In [29]:
# Agrupar por familia y semana, y calcular la suma de 'total', 'base_imponible' y unidades_vendidas
def agrupar_por_semana_y_familia(df: pd.DataFrame) -> pd.DataFrame:
    """
    Agrupa el DataFrame por familia y semana, y calcula la suma de 'total', 'base_imponible' y 'unidades_vendidas'.

    Args:
        df (pd.DataFrame): DataFrame con las columnas 'fecha', 'familia', 'total', 'base_imponible' y 'unidades_vendidas'.

    Returns:
        pd.DataFrame: DataFrame agrupado por familia y semana con las sumas de las columnas especificadas.
    """
    # Validar que el índice sea de tipo DatetimeIndex, si no, intentar convertir la columna 'fecha' y establecerla como índice
    if not isinstance(df.index, pd.DatetimeIndex):
        if 'fecha' in df.columns:
            df = df.copy()
            df['fecha'] = pd.to_datetime(df['fecha'])
            df.set_index('fecha', inplace=True)
        else:
            raise ValueError("El DataFrame debe tener un índice de tipo DatetimeIndex o una columna 'fecha'.")
    
    # Agrupar por semana y familia usando el índice DatetimeIndex (por semana natural)
    df_grouped = df.copy()
    df_grouped['semana'] = df_grouped.index.to_period('W').to_timestamp()
    df_grouped = df_grouped.groupby(['semana', 'familia'])[['total', 'base_imponible', 'unidades_vendidas']].sum().reset_index()
    return df_grouped

df_fleca_semana_familia = agrupar_por_semana_y_familia(df_fleca)
print("DataFrame agrupado por familia y semana:")
print(df_fleca_semana_familia.head())

DataFrame agrupado por familia y semana:
      semana     familia   total  base_imponible  unidades_vendidas
0 2023-01-02    AÑADIDOS    9.60            8.81              33.00
1 2023-01-02      BEBIDA  157.55          143.01              98.00
2 2023-01-02     BEBIDAS  357.45          324.95             149.00
3 2023-01-02  BOCADILLOS  950.40          864.03             380.00
4 2023-01-02    BOLLERIA  907.13          825.11             631.89


In [32]:
# Almacenar el DataFrame agrupado en un archivo parquet en la carpeta interim (datos intermedios)
carpeta = Path("c:/Workspace/mlops_fleca_project/data")
df_fleca_semana_familia.to_parquet(carpeta / "interim" / "df_fleca_semana_family", index=False)
print(df_fleca_semana_familia.head())

      semana     familia   total  base_imponible  unidades_vendidas
0 2023-01-02    AÑADIDOS    9.60            8.81              33.00
1 2023-01-02      BEBIDA  157.55          143.01              98.00
2 2023-01-02     BEBIDAS  357.45          324.95             149.00
3 2023-01-02  BOCADILLOS  950.40          864.03             380.00
4 2023-01-02    BOLLERIA  907.13          825.11             631.89


In [9]:
# Agrupamos por mes y sumamos las columnas 'total', 'base_imponible' y 'unidades_vendidas'
# y reiniciamos el índice
# y convertimos la columna 'mes' a formato Timestamp    

def agrupar_por_mes_y_sumar(df: pd.DataFrame) -> pd.DataFrame:
    """
    Agrupa el DataFrame por la columna 'mes' y suma las columnas 'total', 'base_imponible'
    y 'unidades_vendidas'. Además, reinicia el índice y convierte la columna 'mes' a formato Timestamp.

    Args:
        df (pd.DataFrame): DataFrame que contiene la columna 'mes' (de tipo Period) y las columnas
                           'total', 'base_imponible' y 'unidades_vendidas'.

    Returns:
        pd.DataFrame: DataFrame agrupado por mes con las sumas correspondientes.
    """
    df_grouped = df.groupby(df['mes'])[['total', 'base_imponible', 'unidades_vendidas']].sum()
    df_grouped.reset_index(inplace=True)
    df_grouped['mes'] = df_grouped['mes'].dt.to_timestamp()
    return df_grouped


df_fleca_monthly = agrupar_por_mes_y_sumar(df_fleca)
print(df_fleca_monthly.head())



         mes     total  base_imponible  unidades_vendidas
0 2023-01-01  20361.41        18636.70           11625.88
1 2023-02-01  17444.14        15959.63           10205.77
2 2023-03-01  21301.03        19475.63           12286.30
3 2023-04-01  28905.01        26450.90           16008.64
4 2023-05-01  25389.64        23209.71           14379.80


In [11]:
# Almacenar el DataFrame agrupado en un archivo parquet en la carpeta interim (datos intermedios)

carpeta = Path("D:/Workspace/mlops_fleca_project/data")
(carpeta / "interim").mkdir(parents=True, exist_ok=True)
df_fleca_monthly.to_parquet(carpeta / "interim" / "df_fleca_monthly", index=False)

A partir de aquí, ya no sirve (lo dejo como histórico por si lo necesito para ver algo más adelante)

In [12]:
import pandas as pd

def agrupar_por_mes_y_familia(df: pd.DataFrame) -> pd.DataFrame:
    """
    Agrupa el DataFrame por 'mes' y 'familia', sumando las columnas 'total', 'base_imponible'
    y 'unidades_vendidas'. Reinicia el índice y convierte la columna 'mes' de Period a Timestamp.

    Args:
        df (pd.DataFrame): DataFrame que contiene las columnas 'mes', 'familia', 'total',
                           'base_imponible' y 'unidades_vendidas'. Se asume que la columna 'mes'
                           es de tipo Period.

    Returns:
        pd.DataFrame: DataFrame agrupado por 'mes' y 'familia' con las sumas correspondientes.
    """
    df_grouped = df.groupby(['mes', 'familia'])[['total', 'base_imponible', 'unidades_vendidas']].sum()
    df_grouped.reset_index(inplace=True)
    df_grouped['mes'] = df_grouped['mes'].dt.to_timestamp()
    return df_grouped

df_fleca_monthly_family = agrupar_por_mes_y_familia(df_fleca)
print(df_fleca_monthly_family.head())




         mes     familia    total  base_imponible  unidades_vendidas
0 2023-01-01    AÑADIDOS    37.15           34.06             107.00
1 2023-01-01      BEBIDA   624.60          567.09             378.00
2 2023-01-01     BEBIDAS  1347.90         1225.37             552.00
3 2023-01-01  BOCADILLOS  3560.20         3236.15            1374.00
4 2023-01-01    BOLLERIA  3350.27         3047.58            2237.88


In [14]:
# Almacenar el DataFrame agrupado en un archivo parquet en la carpeta interim (datos intermedios)

carpeta = Path("D:/Workspace/mlops_fleca_project/data")
df_fleca_monthly_family.to_parquet(carpeta / "interim" / "df_fleca_monthly_family", index=False)
print(df_fleca_monthly_family.head())


         mes     familia    total  base_imponible  unidades_vendidas
0 2023-01-01    AÑADIDOS    37.15           34.06             107.00
1 2023-01-01      BEBIDA   624.60          567.09             378.00
2 2023-01-01     BEBIDAS  1347.90         1225.37             552.00
3 2023-01-01  BOCADILLOS  3560.20         3236.15            1374.00
4 2023-01-01    BOLLERIA  3350.27         3047.58            2237.88
